In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(singscore))

source(file.path("../3.bulk-signatures/utils", "singscore_utils.R"))

In [9]:
seed <- 1234
num_permutations <- 1000
dataset <- "bortezomib"

data_dir <- "data"
data_file <- file.path(data_dir, paste0(dataset, "_signature_analytical_set.tsv.gz"))

input_results_dir <- file.path("results", "signatures")
signature_file <- file.path(input_results_dir, paste0("signature_summary_", dataset, "_signature.tsv.gz"))
tukey_file <- file.path(input_results_dir, paste0("tukey_results_", dataset, "_signature.tsv.gz"))

output_dir <- file.path("results", "singscore")
output_results_file <- file.path(output_dir, paste0("singscore_results", dataset, ".tsv.gz"))

In [3]:
set.seed(seed)

In [4]:
# Load profiles
bulk_col_types <- readr::cols(
    .default = readr::col_double(),
    Metadata_Plate = readr::col_character(),
    Metadata_Well = readr::col_character(),
    Metadata_cell_count = readr::col_integer(),
    Metadata_batch = readr::col_character(),
    Metadata_clone_number = readr::col_character(),
    Metadata_plate_map_name = readr::col_character(),
    Metadata_treatment = readr::col_character(),
    Metadata_dataset = readr::col_character(),
    Metadata_clone_type = readr::col_character(),
    Metadata_clone_type_indicator = readr::col_character(),
    Metadata_model_split = readr::col_character(),
    Metadata_cell_density = readr::col_character(),
    Metadata_treatment_time = readr::col_character(),
    Metadata_unique_sample_name = readr::col_character(),
    Metadata_time_to_adhere = readr::col_character()
)

data_df <- readr::read_tsv(data_file, col_types = bulk_col_types)

print(dim(data_df))
head(data_df, 4)

[1]  405 3546


Metadata_Plate,Metadata_Well,Metadata_batch,Metadata_cell_count,Metadata_cell_density,Metadata_celltype_shorthand_from_plate_graph,Metadata_clone_number,Metadata_date,Metadata_plate_map_name,Metadata_time_to_adhere,⋯,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_20_00,Nuclei_Texture_Variance_RNA_20_01,Nuclei_Texture_Variance_RNA_20_02,Nuclei_Texture_Variance_RNA_20_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
219907,B02,2021_03_03_Batch12,6139,2.5x10^3 cells/well,1,WT_parental,20210205,219814,48 hr,⋯,-0.6430638,-0.6342366,-0.6554599,-0.6673215,-0.6546287,-0.6977682,-0.6320059,-0.6297243,-0.6345844,-0.6329460
219907,B03,2021_03_03_Batch12,4567,2.5x10^3 cells/well,2,CloneA,20210205,219814,48 hr,⋯,-0.8308873,-0.8344854,-0.8304672,-0.8178338,-0.8309076,-0.8234369,-0.8347194,-0.8332018,-0.8327623,-0.8296075
219907,B04,2021_03_03_Batch12,5624,2.5x10^3 cells/well,3,CloneE,20210205,219814,48 hr,⋯,-0.9813265,-0.9759700,-0.9728225,-0.9617426,-0.9721434,-0.9618017,-0.9802334,-0.9790954,-0.9839611,-0.9793490
219907,B05,2021_03_03_Batch12,5894,2.5x10^3 cells/well,4,WT clone 01,20210205,219814,48 hr,⋯,-1.0862537,-1.0826744,-1.0689886,-1.0663278,-1.0579078,-1.0610340,-1.0902747,-1.0863724,-1.0886331,-1.0853202


In [10]:
# Load signatures
sig_col_types <- readr::cols(
    features = readr::col_character(),
    non_specific_exclude = readr::col_logical(),
    final_signature = readr::col_logical(),
    dataset = readr::col_character()
)

signature_df <- readr::read_tsv(signature_file, col_types = sig_col_types)

print(dim(signature_df))
head(signature_df, 4)

[1] 3434    6


features,non_status_significant_exclude,cell_count_exclude,non_specific_exclude,final_signature,dataset
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
Cells_AreaShape_Area,FALSE,TRUE,TRUE,FALSE,bortezomib
Cells_AreaShape_Center_X,TRUE,FALSE,FALSE,FALSE,bortezomib
Cells_AreaShape_Center_Y,TRUE,FALSE,FALSE,FALSE,bortezomib
Cells_AreaShape_Compactness,TRUE,FALSE,TRUE,FALSE,bortezomib


In [11]:
signature_df %>% dplyr::filter(final_signature)

features,non_status_significant_exclude,cell_count_exclude,non_specific_exclude,final_signature,dataset
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
Cells_Correlation_K_DNA_AGP,FALSE,FALSE,FALSE,TRUE,bortezomib
Cells_Correlation_RWC_AGP_RNA,FALSE,FALSE,FALSE,TRUE,bortezomib
Cells_Correlation_RWC_ER_Mito,FALSE,FALSE,FALSE,TRUE,bortezomib
Cells_Intensity_MeanIntensityEdge_Mito,FALSE,FALSE,FALSE,TRUE,bortezomib
Cytoplasm_Correlation_K_Mito_DNA,FALSE,FALSE,FALSE,TRUE,bortezomib
Cytoplasm_Correlation_Manders_DNA_AGP,FALSE,FALSE,FALSE,TRUE,bortezomib
Cytoplasm_Correlation_RWC_DNA_Mito,FALSE,FALSE,FALSE,TRUE,bortezomib
Cytoplasm_Granularity_14_ER,FALSE,FALSE,FALSE,TRUE,bortezomib
Cytoplasm_Granularity_15_Mito,FALSE,FALSE,FALSE,TRUE,bortezomib


In [12]:
# Load Tukey results (to determine if feature is "up" or "down")
tukey_cols <- readr::cols(
    term = readr::col_character(),
    comparison = readr::col_character(),
    estimate = readr::col_double(),
    conf.low = readr::col_double(),
    conf.high = readr::col_double(),
    adj.p.value = readr::col_double(),
    feature = readr::col_character(),
    neg_log_adj_p = readr::col_double(),
    dataset = readr::col_character()
)

tukey_df <- readr::read_tsv(tukey_file, col_types = tukey_cols)

print(dim(tukey_df))
head(tukey_df, 4)

[1] 291890      9


term,comparison,estimate,conf.low,conf.high,adj.p.value,feature,neg_log_adj_p,dataset
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
Metadata_clone_type_indicator,1-0,0.11148995,-0.003859527,0.2268394,0.05808988,Nuclei_RadialDistribution_FracAtD_Mito_3of4,1.235899493,bortezomib
Metadata_batch,2021_03_03_Batch13-2021_03_03_Batch12,-0.02885406,-0.243773017,0.1860649,0.98548295,Nuclei_RadialDistribution_FracAtD_Mito_3of4,0.006350883,bortezomib
Metadata_batch,2021_03_03_Batch15-2021_03_03_Batch12,0.11550730,-0.096317972,0.3273326,0.49250742,Nuclei_RadialDistribution_FracAtD_Mito_3of4,0.307587224,bortezomib
Metadata_batch,2021_03_05_Batch16-2021_03_03_Batch12,0.06492660,-0.149992362,0.2798456,0.86206307,Nuclei_RadialDistribution_FracAtD_Mito_3of4,0.064460957,bortezomib


In [13]:
# Subset data to process dataset-specific signature
signature_subset_df <- signature_df %>%
    dplyr::filter(dataset == !!dataset, final_signature)

tukey_subset_df <- tukey_df %>%
    dplyr::filter(
        dataset == !!dataset,
        term == "Metadata_clone_type_indicator",
        feature %in% signature_subset_df$features
    )

# Ensure that the comparison is always resistant vs. senstive
# and never the other way around!
stopifnot(length(table(tukey_subset_df$comparison)) == 1)

# Determine feature direction
up_features <- tukey_subset_df %>% dplyr::filter(estimate > 0) %>% dplyr::pull(feature)
down_features <- tukey_subset_df %>% dplyr::filter(estimate < 0) %>% dplyr::pull(feature)

# Store signature for downstream analyses
signature_features <- list("up" = up_features, "down" = down_features)

In [14]:
signature_features

$up
[1] "Nuclei_AreaShape_Zernike_6_0"                        
[2] "Cytoplasm_Texture_Entropy_Mito_20_01"                
[3] "Cytoplasm_Texture_Entropy_Mito_20_03"                
[4] "Cytoplasm_Texture_InfoMeas1_RNA_5_01"                
[5] "Cells_Correlation_K_DNA_AGP"                         
[6] "Cytoplasm_Granularity_14_ER"                         
[7] "Cytoplasm_Texture_Entropy_Mito_20_02"                
[8] "Cytoplasm_Granularity_15_Mito"                       
[9] "Cytoplasm_Texture_InverseDifferenceMoment_Mito_20_01"

$down
 [1] "Nuclei_AreaShape_Zernike_8_6"          
 [2] "Cytoplasm_Texture_InfoMeas2_Mito_20_02"
 [3] "Cytoplasm_Texture_InfoMeas2_ER_5_00"   
 [4] "Cytoplasm_Texture_InfoMeas2_Mito_20_03"
 [5] "Cytoplasm_Texture_InfoMeas2_Mito_20_01"
 [6] "Nuclei_AreaShape_Zernike_2_2"          
 [7] "Cytoplasm_Texture_InfoMeas2_Mito_10_02"
 [8] "Cytoplasm_Texture_InfoMeas2_Mito_10_00"
 [9] "Nuclei_Texture_Correlation_Mito_5_00"  
[10] "Nuclei_AreaShape_Zernike_9_3"          
[11] "Cytoplasm_Correlation_Manders_DNA_AGP" 
[12] "Cytoplasm_Correlation_RWC_DNA_Mito"    
[13] "Cytoplasm_Texture_InfoMeas2_Mito_5_01" 
[14] "Cells_Correlation_RWC_ER_Mito"         
[15] "Cells_Intensity_MeanIntensityEdge_Mito"
[16] "Nuclei_Texture_Correlation_Mito_5_01"  
[17] "Nuclei_Texture_Correlation_Mito_5_02"  
[18] "Cytoplasm_Texture_InfoMeas2_Mito_5_03" 
[19] "Nuclei_Texture_Correlation_Mito_5_03"  
[20] "Nuclei_Texture_Correlation_Mito_10_00" 
[21] "Nuclei_Intensity_MinIntensity_AGP"     
[22] "Cytoplasm_Texture_InfoMeas2_Mito_5_00" 
[23] "Cells_Correlation_RWC_AGP_RNA"         
[24] "Cytoplasm_Texture_InfoMeas2_Mito_5_02" 
[25] "Cytoplasm_Correlation_K_Mito_DNA"

In [15]:
singscore_output = singscorePipeline(
    df = data_df,
    sig_feature_list = signature_features,
    num_permutations = num_permutations
)

full_results_df <- singscore_output[["results"]]
permuted <- singscore_output[["permuted"]]

# Get max and minimum values of permutation results
min_val <- quantile(as.vector(as.matrix(permuted)), 0.05)
max_val <- quantile(as.vector(as.matrix(permuted)), 0.95)

# Annotate some key metadata and store to list
sing_score_results_df <- full_results_df %>%
    dplyr::mutate(
        dataset = dataset,
        min_permuted_value = min_val,
        max_permuted_value = max_val
    )

Warning message:
“'tidy.numeric' is deprecated.
See help("Deprecated")”Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”

In [16]:
sing_score_results_df %>% readr::write_tsv(output_results_file)

print(dim(sing_score_results_df))
head(sing_score_results_df)

[1] 405  28


Metadata_Plate,Metadata_Well,Metadata_batch,Metadata_cell_count,Metadata_cell_density,Metadata_celltype_shorthand_from_plate_graph,Metadata_clone_number,Metadata_date,Metadata_plate_map_name,Metadata_time_to_adhere,⋯,TotalScore,TotalDispersion,UpScore,UpDispersion,DownScore,DownDispersion,Metadata_permuted_p_value,dataset,min_permuted_value,max_permuted_value
<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
219907,B02,2021_03_03_Batch12,6139,2.5x10^3 cells/well,1,WT_parental,20210205,219814,48 hr,⋯,-0.22817175,2882.1744,-0.12353573,1598.2428,-0.104636026,1283.9316,0.977,bortezomib,-0.1877069,0.1892476
219907,B03,2021_03_03_Batch12,4567,2.5x10^3 cells/well,2,CloneA,20210205,219814,48 hr,⋯,0.09777312,825.8082,0.13518676,545.5968,-0.037413645,280.2114,0.188,bortezomib,-0.1877069,0.1892476
219907,B04,2021_03_03_Batch12,5624,2.5x10^3 cells/well,3,CloneE,20210205,219814,48 hr,⋯,0.24496199,1498.9086,0.15668908,539.6664,0.088272909,959.2422,0.014,bortezomib,-0.1877069,0.1892476
219907,B05,2021_03_03_Batch12,5894,2.5x10^3 cells/well,4,WT clone 01,20210205,219814,48 hr,⋯,0.13694715,1676.8206,0.14491175,849.5298,-0.007964602,827.2908,0.113,bortezomib,-0.1877069,0.1892476
219907,B06,2021_03_03_Batch12,1277,2.5x10^3 cells/well,5,WT clone 02,20210205,219814,48 hr,⋯,-0.38491359,1478.1522,-0.25936977,524.8404,-0.125543820,953.3118,1.000,bortezomib,-0.1877069,0.1892476
219907,B07,2021_03_03_Batch12,5452,2.5x10^3 cells/well,1,WT_parental,20210205,219814,48 hr,⋯,-0.18470935,2229.8304,-0.06158631,877.6992,-0.123123037,1352.1312,0.939,bortezomib,-0.1877069,0.1892476
